# Cutting Stock 

Cada rollo de papel tiene un ancho de 100

Ancho / Demanda:

- 14 / 211
- 31 / 395
- 36 / 610
- 45 / 97

¿Cómo cortar cada rollo de paper para que el número de rollos originales usados sea el menor posible?


### Veamos como formularlo en Jump
**OJO: En este módulo usaremos modelos enteros (que se resuelven con ``Cbc``) y continuos (que se resuelven con ``Clp``)**

In [ ]:
using JuMP
using Clp 
using Cbc


Datos del problema (de juguete, despues probaremos con los correctos):

In [ ]:
#anchos
w=[14 31 36 45]
requerimientos=[2; 4; 6; 1]

## Ejercicio: Resolver el modelo con la formulación tradicional (usar Cbc para resolverlo)
Recuerdo: Para definir una variable entera o binaria usamos:

In [ ]:
K = sum(requerimientos)
model = Model(solver=CbcSolver())
@variable(model, x[1:length(requerimientos),1:K] >= 0, Int)
### COMPLETAR
model




In [ ]:
solve(model)
@show getobjectivevalue(model)
@show getvalue(x)

# Generación de Columnas

Inicializamos el ***"problema master"*** con solo 2 variables, de forma de tener una solución inicial:
- ancho (14,31,36,45), cantidad (1,1,0,1), como $x_1$
- ancho (14,31,36,45), cantidad (0,0,2,0), como $x_2$

**\[Recuerdo 1\]** ¿Qué significa cada variable? _El número de rollos de papel que cortaremos con este patrón._

**\[Recuerdo 2\]** ¿Como se ve el problema master?

$$
\begin{align}
\nonumber\min\qquad\qquad\quad &x_1+x_2 \\
s.t.\qquad\left( \begin{array}{c}
1 \\
1 \\
0 \\
1 \end{array} \right)&x_1+\left( \begin{array}{c}
0 \\
0 \\
2 \\
0 \end{array} \right)x_2 \ge \left( \begin{array}{c}
211 \\
395 \\
610 \\
97 \end{array} \right) \\ 
&x_1,x_2\ge0
\end{align}$$

In [ ]:
#anchos
w=[14 31 36 45]
requerimientos=[211; 395; 610; 97]
patrones = [[1,1,0,1],[0,0,2,0]]

In [ ]:
master = Model(solver=ClpSolver())  

#una variable por cada patrón
@variable(master, x[1:length(patrones)] >= 0)

#definimos una referencia a las restricciones para recuperar los duales.
@defConstrRef myCons[1:4]
for i=1:4
    myCons[i] =  ### COMPLETAR
end

#Objetivo
@objective(master, Min, sum(x))
master

In [ ]:
status=solve(master)
@show getValue(x)

#obtenemos el optimo
println("\nSolución óptima:\n")

println("anchos: ", w)

epsilon=1e-6

for i=1:length(patrones)
   
    if getValue(x[i])>epsilon 
        println("Patron actual: ", patrones[i], ", Número de rollos con este patrón: ", getValue(x[i]))
    end
end

Necesitamos con ese patrón 700 rollos.  ¿Óptimo?  Claramente no. 

Generemos entonces nuevos patrones basados en los costos reducidos. Denotemos $r=(r_1,r_2,r_3,r_4)$ como los optimos duales de las restricciones 1, 2, 3, 4. El costo reducido de una variable $x_k$, con patrón $A_k$ puede calcularse como
$$rc(x_k)=1-A_k^Tr$$

Queremos buscar una variable $x_k$ tal que $rc(x_k)<0$, lo que podemos hacer resolviendo el siguiente problema de la mochila:

$$\begin{align}
z^*=\max\qquad &r_1a_{k,1}+r_2a_{k,2}+r_3a_{k,3}+r_4a_{k,4} \\
s.t.\qquad &14a_{k,1}+31a_{k,2}+36a_{k,3}+45a_{k,4}\le 100 \\
&a_{k,1},a_{k,2},a_{k,3},a_{k,4}\ge0,~\textrm{y enteros}
\end{align}$$

Si $z^*>1$, entonces $x_k$ con un patrón de corte $(a_{k,1},a_{k,2},a_{k,3},a_{k,4})$ debe ser agregado al _master_ y re-resolver el problema.

*OJO:* como este modelo es entero, debemos usar Cbc como solver esta vez.

In [ ]:
duales=getDual(myCons)[1:4]
sub = Model(solver=CbcSolver())  

#variable que define el patrón. Esta es entera.
@variable(sub, a[1:4]>=0, Int)

# completar el modelo

sub



In [ ]:
status=solve(sub)


nuevoPatron=getValue(a)[1:4]
println("Costo reducido: ", 1-getobjectivevalue(sub))

println("Anchos: ", w)

nuevoPatron = round(nuevoPatron)

println("\nNuevo patrón de corte: ", nuevoPatron)

push!(patrones, nuevoPatron)


El costo reducido es $(1-3)=-2<0$. Agregamos esta variable entonces al ***"master problem"***.

In [ ]:
master

JuMP (y casi todos los solvers de optimización) permiten definir un modelo tanto por filas (es decir, restricción por restricción), como por columa (es decir, variable por variable). De esta forma, podemos agregar facilmente una nueva variable en este caso, simplemente necesitamos decir:

- el objetivo de la variable
- en que restricciones aparece la variable, y con que coeficientes

In [ ]:
@variable(master, z>=0, objective=1, inconstraints=myCons, coefficients=nuevoPatron)
master

In [ ]:
master

In [ ]:
#Resolvermos
status=solve(master)

#get the optimal solution
println("\nSolución óptima:\n")

println("Anchos: ", w)

for i=1:length(x)
   
    if getValue(x[i])>epsilon
        println("Patron inicial: ", patrones[i], ", Número de rollos con este patrón: ", getValue(x[i]))


    end
end


if getValue(z)>epsilon
    println("Patron nuevo: ", nuevoPatron, ", Número de rollos con este patrón: ", getValue(z))
end


Como vemos, el objetivo se reduce a 577.3 gracias a la nueva variable. 



# Pongamos ahora todo junto! (Sugerencia: reiniciar el Kernel)

Sugerencia: cree un arreglo de variables ``variables`` para poder referirse a ellas de forma mas facil. Para esto, puede crear el arreglo e ir agregando las variables que defina:

```
variables=Variable[]
push!(variables,x[1])
push!(variables,x[2])
push!(variables, z)
```



In [ ]:
using JuMP  
using Clp
using Cbc
w=[14 31 36 45]
requerimientos=[211; 395; 610; 97]
patrones = [[1,1,0,1],[0,0,2,0]]


## Completar

Excelente! Necesitamos entonces solo 3 patrones de cortes (y que son muy distintos a los que inicialmente mencionamos)

### Como obtener una solución ENTERA? ###

Generación de columna solo resuelve problemas lineales LP. Sin embargo, el problema es entero (no podemos usar 206.25 rollos  con el primer patrón). ¿Se puede hacer una generación de columnas _entera_? 

*Si*, pero eso requiere una técnica avanzada llamada  [branch-and-price](http://en.wikipedia.org/wiki/Branch_and_price) que es básicamente un árbol de branch-and-bound donde en cada nodo resuelvo un problema de generación de columnas. Desafortunadamente, los solvers comerciales (Gurobi, CPLEX) no resuelven este tipo de problemas. Hasta ahora, el único solver académico que resuelve branch-and-price es [SCIP](http://scip.zib.de/).

En vez de resolver el problema entero a optimalidad, podemos usar dos métodos para generar una solución entera.


#### Metodo 1: Redondeo ####

Una heuristica común es redondear la solución fraccionaria al entero mas _cercano_ y _factible_. Como redondear dependerá del problema. En este problema, debemos redondear _hacia arriba_ para poder estar seguros de que la solución es factible. 

In [ ]:
println("\nSolución entera basada en redondeo:\n")

println("Ancho: ", w)

suma=0.0

for i=1:length(variables)
    if getValue(variables[i])>epsilon
        println("Patron: ", patrones[i], ", Número de rollos con este patrón: ", ceil(getValue(variables[i])))
        suma=suma+ceil(getValue(variables[i]))
    end
end


println("Número total de rollos requeridos: ", suma)

Ok, tenemos una solución entera que usa 454 rollos en total. Recordemos que la solución LP nos da valor 452.25. O sea, aunque sea una heurística, sabemos que estamos a menos de un 0.4% del óptimo.

#### Método 2: Branch-and-Bound con las columnas generadas ####

Otra práctica es usar directamente las columnas generadas pero forzando que sean enteras. Esto no entrega un óptimo, ya que otra columna podría agregarse, pero puede entregar una  solución suficientemente buena.



OJO: Por ser ahora un problema entero, debemos re-formularlo con Cbc como solver

Efectivamente, podiamos ahorrarnos 1 rollo extra usando el método 2.